In [ ]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 33.0 MB/s eta 0:00:00


In [ ]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.7 MB/s eta 0:00:00


In [ ]:
import os
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM , PreTrainedTokenizerFast, AdamW, BitsAndBytesConfig, GPT2LMHeadModel, LlamaTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd
import tqdm
import urllib.request
import bitsandbytes
import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_dir='/content/drive/MyDrive/DBDBDeep/model_skt2'

In [ ]:
# 모델 및 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained(model_dir, bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = GPT2LMHeadModel.from_pretrained(model_dir)

In [ ]:
# 모델을 GPU로 옮기기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
text = '벽지가 벌어졌어'

In [ ]:
sent = '<usr>' + text + '<sys>'

In [ ]:
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = torch.tensor([input_ids], dtype=torch.long).to(device)

In [ ]:
output = model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id)

In [ ]:
decoded_sentence = tokenizer.decode(output[0].tolist())

In [ ]:
output = model.generate(input_ids, max_length=50, do_sample=True, top_k=10)
tokenizer.decode(output[0].tolist())

'</s><usr> 벽지가 벌어졌어<sys> 벽지가 벌어졌는데 벽지가 벌어졌다면, 보통 수리를 통해 문제를 해결해야 합니다.</s>'

In [ ]:
decoded_sentence.split('<sys> ')[1].replace('</s>', '')

'벽지가 벌어졌을 때 주변 환경에 어떤 영향을 미칠까??'

In [ ]:
# 사용자 입력을 받아 챗봇이 응답을 반환하도록 함수 만들기

def return_answer_by_chatbot(user_text):
    sent = '<usr>' + user_text + '<sys>'
    input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent, add_special_tokens=False)   # 입력 문장을 인코딩
    input_ids = torch.tensor([input_ids], dtype=torch.long).to(device)  # 텐서로 변환하고 GPU로 이동합니다.

    # 모델을 사용하여 챗봇의 응답을 생성합. 최대 길이는 50, 샘플링을 수행하며 상위 2개 토큰을 선택.
    output = model.generate(input_ids, max_length=150, do_sample=True, top_k=2)
    sentence = tokenizer.decode(output[0].tolist())  # 토큰을 문장으로 디코딩
    chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
    return chatbot_response

In [ ]:
return_answer_by_chatbot('도배할때 주의사항이 뭐야')

'도배 할 때는 벽의 표면을 철저히 점검하여 부식, 균열, 또는 기타 결함이 있는지 확인해야 합니다. 필요한 경우 해당 결함을 보정하고, 결함을 정확히 파악하여 적절한 조치를 취해야 합니다.'

In [ ]:
return_answer_by_chatbot('도배비용 알려줘')

'일반적으로 도배 견적은 (도배평수/5*도배지가격)+인건비+부자재로 산출합니다.'

In [ ]:
return_answer_by_chatbot('벽지 예쁘게 붙이는 법 알려줘')

'벽지면에 길 형태의 타원으로 발생한 기포는 자주 나타나나요?'